In [2]:
import pandas as pd
import MetaTrader5 as mt5
import matplotlib.pyplot as plt
import json

In [3]:
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

tierminal_info = mt5.terminal_info()._asdict()
version= mt5.version()
print("MetaTrader5 version:", version)
df=pd.DataFrame(list(tierminal_info.items()),columns=['property','value'])
print(df)


MetaTrader5 version: (500, 4620, '11 Oct 2024')
                 property                                              value
0       community_account                                              False
1    community_connection                                              False
2               connected                                               True
3            dlls_allowed                                              False
4           trade_allowed                                               True
5       tradeapi_disabled                                              False
6           email_enabled                                              False
7             ftp_enabled                                              False
8   notifications_enabled                                              False
9                    mqid                                               True
10                  build                                               4620
11                maxbars   

In [4]:
account_info = mt5.account_info()._asdict()
df_account_info = pd.DataFrame(list(account_info.items()), columns=['property', 'value'])
print(df_account_info)

              property              value
0                login          312241304
1           trade_mode                  0
2             leverage               1000
3         limit_orders                300
4       margin_so_mode                  0
5        trade_allowed               True
6         trade_expert               True
7          margin_mode                  2
8      currency_digits                  2
9           fifo_close              False
10             balance           10022.72
11              credit                0.0
12              profit                0.0
13              equity           10022.72
14              margin                0.0
15         margin_free           10022.72
16        margin_level                0.0
17      margin_so_call               50.0
18        margin_so_so               20.0
19      margin_initial                0.0
20  margin_maintenance                0.0
21              assets                0.0
22         liabilities            

In [5]:
symbols=mt5.symbols_total()
print("Total symbols:",symbols)

Total symbols: 1505


In [6]:
symbols=mt5.symbols_get()

count=0
for s in symbols:
    count+=1
    print("{}. {}".format(count,s.name))
    if count==5: break
print()

1. EURUSD
2. GBPUSD
3. USDCHF
4. USDJPY
5. USDCAD



In [7]:
# get symbols containing RU in their names
ru_symbols=mt5.symbols_get("*RU*")
print('len(*RU*): ', len(ru_symbols))
for s in ru_symbols:
    print(s.name)
print()
 

len(*RU*):  2
EURUSD
NEARUSD



In [8]:
# get symbols whose names do not contain USD, EUR, JPY and GBP
group_symbols=mt5.symbols_get(group="*,!*USD*,!*EUR*,!*JPY*,!*GBP*")
print('len(*,!*USD*,!*EUR*,!*JPY*,!*GBP*):', len(group_symbols))
# for s in group_symbols:
#     print(s.name,":",s)

len(*,!*USD*,!*EUR*,!*JPY*,!*GBP*): 1396


In [9]:
symbol_info=mt5.symbol_info("EURJPY")
if symbol_info!=None:
    # display the terminal data 'as is'    
    df=pd.DataFrame(list(symbol_info._asdict().items()),columns=['property','value'])
    print(df)

         property                         value
0          custom                         False
1      chart_mode                             0
2          select                         False
3         visible                         False
4   session_deals                             0
..            ...                           ...
91        formula                              
92           isin                              
93           name                        EURJPY
94           page                              
95           path  Forex\Standard\Majors\EURJPY

[96 rows x 2 columns]


In [10]:
selected=mt5.symbol_select("GBPUSD",True)
print("GBPUSD selected=",selected)

GBPUSD selected= True


In [11]:
lasttick=mt5.symbol_info_tick("GBPUSD")
df=pd.DataFrame(lasttick._asdict(),index=[0])
df['time']=pd.to_datetime(df['time'],unit='s')
print(df)


                 time      bid      ask  last  volume       time_msc  flags  \
0 2024-11-26 14:53:12  1.26081  1.26105   0.0       0  1732632792360   1122   

   volume_real  
0          0.0  


In [12]:
import pytz
from datetime import datetime

pd.set_option('display.max_columns', 20) # number of columns to be displayed
pd.set_option('display.width', 1500)      # max table width to display

timezone = pytz.timezone("Etc/UTC")
utc_from = datetime(2020, 1, 10, tzinfo=timezone)
rates = mt5.copy_rates_from("EURUSD", mt5.TIMEFRAME_H4, utc_from, 10)

for rate in rates:
    print(rate)

(1578484800, 1.11377, 1.11378, 1.11104, 1.11192, 6699, 16, 0)
(1578499200, 1.11192, 1.11301, 1.11081, 1.11172, 7627, 16, 0)
(1578513600, 1.1117, 1.1117, 1.11009, 1.11042, 3412, 16, 0)
(1578528000, 1.11043, 1.11186, 1.11022, 1.11166, 2511, 16, 0)
(1578542400, 1.11166, 1.11182, 1.11119, 1.11175, 1770, 16, 0)
(1578556800, 1.11174, 1.11196, 1.10976, 1.10986, 5889, 16, 0)
(1578571200, 1.10986, 1.11166, 1.10958, 1.11142, 5392, 16, 0)
(1578585600, 1.1114, 1.1114, 1.10918, 1.11039, 5525, 16, 0)
(1578600000, 1.11039, 1.1109, 1.11026, 1.11037, 2657, 16, 0)
(1578614400, 1.11035, 1.11085, 1.10993, 1.11034, 1787, 16, 0)


In [13]:
rates_frame = pd.DataFrame(rates)
rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
print(rates_frame)

                 time     open     high      low    close  tick_volume  spread  real_volume
0 2020-01-08 12:00:00  1.11377  1.11378  1.11104  1.11192         6699      16            0
1 2020-01-08 16:00:00  1.11192  1.11301  1.11081  1.11172         7627      16            0
2 2020-01-08 20:00:00  1.11170  1.11170  1.11009  1.11042         3412      16            0
3 2020-01-09 00:00:00  1.11043  1.11186  1.11022  1.11166         2511      16            0
4 2020-01-09 04:00:00  1.11166  1.11182  1.11119  1.11175         1770      16            0
5 2020-01-09 08:00:00  1.11174  1.11196  1.10976  1.10986         5889      16            0
6 2020-01-09 12:00:00  1.10986  1.11166  1.10958  1.11142         5392      16            0
7 2020-01-09 16:00:00  1.11140  1.11140  1.10918  1.11039         5525      16            0
8 2020-01-09 20:00:00  1.11039  1.11090  1.11026  1.11037         2657      16            0
9 2020-01-10 00:00:00  1.11035  1.11085  1.10993  1.11034         1787      16  

ต่อไปนี้คือลักษณะความแตกต่างระหว่างฟังก์ชันเหล่านี้:

### 1. **`copy_rates_from`**
   - **วัตถุประสงค์**: ฟังก์ชันนี้ใช้ดึงข้อมูลราคาของสัญลักษณ์ที่ระบุ โดยใช้เวลาที่ระบุ (ในกรณีนี้เริ่มต้นจากเวลา `from_time`) และจำนวนแท่งข้อมูล (bars) ที่ต้องการ
   - **การใช้งาน**: ฟังก์ชันนี้จะดึงข้อมูล **price bars** (หรือ OHLC - Open, High, Low, Close) จากช่วงเวลาที่ระบุ
   - **ตัวอย่างการใช้งาน**:
     ```python
     rates = mt5.copy_rates_from(symbol, mt5.TIMEFRAME_M1, from_time, 100)
     ```
   - **พารามิเตอร์**:
     - `symbol`: สัญลักษณ์ (เช่น EURUSD)
     - `timeframe`: ช่วงเวลา (เช่น M1, H1)
     - `from_time`: เวลาที่เริ่มต้น
     - `count`: จำนวนข้อมูลที่ต้องการดึง

### 2. **`copy_rates_from_pos`**
   - **วัตถุประสงค์**: ฟังก์ชันนี้คล้ายกับ `copy_rates_from` แต่จะแตกต่างที่ **จุดเริ่มต้นเป็นตำแหน่ง (position)** แทนการใช้เวลา
   - **การใช้งาน**: ใช้เมื่อคุณต้องการดึงข้อมูลจากตำแหน่งเริ่มต้นใน **history** แทนที่จะใช้เวลา
   - **ตัวอย่างการใช้งาน**:
     ```python
     rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M1, position, 100)
     ```
   - **พารามิเตอร์**:
     - `symbol`: สัญลักษณ์
     - `timeframe`: ช่วงเวลา
     - `position`: ตำแหน่งเริ่มต้น (เช่น 0 คือข้อมูลล่าสุด)
     - `count`: จำนวนแท่งข้อมูล

### 3. **`copy_rates_range`**
   - **วัตถุประสงค์**: ฟังก์ชันนี้ใช้ดึงข้อมูล **ราคา** ในช่วงเวลาที่กำหนด
   - **การใช้งาน**: ใช้เมื่อคุณต้องการดึงข้อมูล **price bars** ตั้งแต่ช่วงเวลาหนึ่งไปจนถึงช่วงเวลาหนึ่ง
   - **ตัวอย่างการใช้งาน**:
     ```python
     rates = mt5.copy_rates_range(symbol, mt5.TIMEFRAME_M1, start_time, end_time)
     ```
   - **พารามิเตอร์**:
     - `symbol`: สัญลักษณ์
     - `timeframe`: ช่วงเวลา
     - `start_time`: เวลาที่เริ่มต้น
     - `end_time`: เวลาที่สิ้นสุด

### 4. **`copy_ticks_from`**
   - **วัตถุประสงค์**: ฟังก์ชันนี้ใช้ดึงข้อมูล **ticks** (ราคาที่เปลี่ยนแปลง) เริ่มจากเวลาที่กำหนด
   - **การใช้งาน**: ใช้ในการดึงข้อมูลราคาที่เปลี่ยนแปลงในลักษณะ **tick** (ไม่ใช่ข้อมูลแบบ bar) ที่มีข้อมูลการเปลี่ยนแปลงราคาในช่วงเวลาหนึ่ง
   - **ตัวอย่างการใช้งาน**:
     ```python
     ticks = mt5.copy_ticks_from(symbol, from_time, 100)
     ```
   - **พารามิเตอร์**:
     - `symbol`: สัญลักษณ์
     - `from_time`: เวลาที่เริ่มต้น
     - `count`: จำนวนข้อมูลที่ต้องการดึง

### 5. **`copy_ticks_range`**
   - **วัตถุประสงค์**: ฟังก์ชันนี้ใช้ดึงข้อมูล **ticks** ในช่วงเวลาที่กำหนด
   - **การใช้งาน**: ใช้เมื่อคุณต้องการดึงข้อมูล **tick** ในช่วงเวลาที่กำหนดระหว่าง `start_time` ถึง `end_time`
   - **ตัวอย่างการใช้งาน**:
     ```python
     ticks = mt5.copy_ticks_range(symbol, start_time, end_time, mt5.COPY_TICKS_ALL)
     ```
   - **พารามิเตอร์**:
     - `symbol`: สัญลักษณ์
     - `start_time`: เวลาที่เริ่มต้น
     - `end_time`: เวลาที่สิ้นสุด
     - `ticks`: สามารถเลือกประเภทของ ticks ที่ต้องการดึง เช่น **`COPY_TICKS_ALL`** หรือ **`COPY_TICKS_INFO`**

---

### สรุปความแตกต่าง:

- **`copy_rates_from`**: ดึงข้อมูลจากเวลาเริ่มต้นและจำนวนแท่งข้อมูลที่ต้องการ
- **`copy_rates_from_pos`**: ดึงข้อมูลจากตำแหน่งเริ่มต้นของข้อมูลที่มี
- **`copy_rates_range`**: ดึงข้อมูลจากช่วงเวลาที่กำหนด
- **`copy_ticks_from`**: ดึงข้อมูล **tick** จากเวลาเริ่มต้น
- **`copy_ticks_range`**: ดึงข้อมูล **tick** จากช่วงเวลาที่กำหนด

การเลือกใช้ฟังก์ชันที่เหมาะสมขึ้นอยู่กับประเภทข้อมูลที่คุณต้องการ (ราคาแบบ bar หรือ tick) และว่าคุณต้องการดึงข้อมูลจากเวลา, ตำแหน่ง, หรือช่วงเวลาใด

หวังว่าคำอธิบายนี้จะช่วยให้คุณเข้าใจความแตกต่างของฟังก์ชันเหล่านี้ครับ! หากยังมีคำถามเพิ่มเติม สามารถถามได้ครับ!

In [14]:
orders=mt5.orders_total()
print("Total orders:",orders)

Total orders: 0


โครงสร้างของคำขอการเทรดใน MQL (MetaTrader Language) ที่ใช้ในการจัดการคำสั่งการเทรดในแพลตฟอร์ม MetaTrader ซึ่งแต่ละฟิลด์ในตารางนี้มีความหมายดังนี้:

1. **action**: ประเภทของการดำเนินการเทรด ค่าในฟิลด์นี้ต้องเป็นหนึ่งในค่าของ `TRADE_REQUEST_ACTIONS` ที่กำหนด
2. **magic**: รหัส ID ของ EA (Expert Advisor) ที่ใช้ในการจัดการคำสั่งเทรด ช่วยในการจัดการและติดตามคำสั่งที่ส่งไป
3. **order**: หมายเลขคำสั่ง (order ticket) ใช้ในการแก้ไขคำสั่งที่รอดำเนินการ
4. **symbol**: ชื่อของเครื่องมือการเทรด (เช่น คู่เงิน) ที่คำสั่งนั้นถูกวางไว้ ฟิลด์นี้ไม่จำเป็นเมื่อแก้ไขคำสั่งหรือปิดตำแหน่ง
5. **volume**: ปริมาณของการเทรดในล็อต ขึ้นอยู่กับประเภทของการดำเนินการคำสั่ง
6. **price**: ราคาที่คำสั่งจะถูกดำเนินการ การตั้งราคาไม่จำเป็นสำหรับคำสั่งประเภทตลาด (market orders)
7. **stoplimit**: ราคาของคำสั่ง Pending Limit ที่จะตั้งเมื่อราคาถึงจุดที่กำหนด
8. **sl (Stop Loss)**: ราคาที่คำสั่ง Stop Loss จะถูกเปิดเมื่อราคาขยับไปในทิศทางที่ไม่ดี
9. **tp (Take Profit)**: ราคาที่คำสั่ง Take Profit จะถูกเปิดเมื่อราคาขยับไปในทิศทางที่ดี
10. **deviation**: ค่าสูงสุดที่ยอมรับได้จากราคาที่ขอ โดยวัดเป็นจุด (points)
11. **type**: ประเภทของคำสั่ง ค่าของฟิลด์นี้ต้องเป็นหนึ่งในค่าของ `ORDER_TYPE`
12. **type_filling**: ประเภทการเติมคำสั่ง ค่าของฟิลด์นี้ต้องเป็นหนึ่งในค่าของ `ORDER_TYPE_FILLING`
13. **type_time**: ประเภทของการหมดอายุคำสั่ง ค่าของฟิลด์นี้ต้องเป็นหนึ่งในค่าของ `ORDER_TYPE_TIME`
14. **expiration**: เวลาหมดอายุของคำสั่ง Pending ซึ่งใช้กับคำสั่งที่มีประเภท `TIME_SPECIFIED`
15. **comment**: ข้อความเพิ่มเติมที่สามารถใส่ได้ในคำสั่ง
16. **position**: หมายเลขของตำแหน่ง (Position ticket) ใช้ในการเปลี่ยนแปลงหรือปิดตำแหน่ง
17. **position_by**: ใช้ในการปิดตำแหน่งโดยการเปิดคำสั่งในทิศทางตรงข้ามกับตำแหน่งที่มีอยู่

การใช้งาน MQL แบบนี้จะช่วยในการจัดการคำสั่งการเทรดอย่างละเอียดและแม่นยำใน MetaTrader

In [31]:
# send order 
symbol = "USDJPY"
price = mt5.symbol_info_tick(symbol).ask
lot = 0.1
point = mt5.symbol_info(symbol).point
deviation = 20
request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_BUY,
    "price": price,
    # "sl": price - 100 * point,
    # "tp": price + 100 * point,
    "deviation": deviation,
    "magic": 234000,
    "comment": "python script open",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,
}

# send a trading request
result = mt5.order_send(request)
# check the execution result
print("1. order_send(): by {} {} lots at {} with deviation={} points".format(symbol,lot,price,deviation));
if result.retcode != mt5.TRADE_RETCODE_DONE:
    print("2. order_send failed, retcode={}".format(result.retcode))
    # request the result as a dictionary and display it element by element
    result_dict=result._asdict()
    for field in result_dict.keys():
        print("   {}={}".format(field,result_dict[field]))
        # if this is a trading request structure, display it element by element as well
        if field=="request":
            traderequest_dict=result_dict[field]._asdict()
            for tradereq_filed in traderequest_dict:
                print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))
    print("shutdown() and quit")
    mt5.shutdown()
    quit()
 
print("2. order_send done, ", result)
print("   opened position with POSITION_TICKET={}".format(result.order))
print("   sleep 2 seconds before closing position #{}".format(result.order))

1. order_send(): by USDJPY 0.1 lots at 153.204 with deviation=20 points
2. order_send done,  OrderSendResult(retcode=10009, deal=274335307, order=273807474, volume=0.1, price=153.204, bid=153.177, ask=153.204, comment='Request executed', request_id=1287851498, retcode_external=0, request=TradeRequest(action=1, magic=234000, order=0, symbol='USDJPY', volume=0.1, price=153.204, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=0, type_filling=1, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))
   opened position with POSITION_TICKET=273807474
   sleep 2 seconds before closing position #273807474


In [23]:
orders=mt5.orders_total()
print("Total orders:",orders)

Total orders: 0


In [18]:
orders=mt5.orders_get(symbol="USDJPY")
for order in orders:
    print(order)

In [32]:
positions_total=mt5.positions_total()
if positions_total>0:
    print("Total positions=",positions_total)
    # get all positions on USDJPY
    positions=mt5.positions_get(symbol="USDJPY")
    df=pd.DataFrame(list(positions),columns=positions[0]._asdict().keys())
    df['time']=pd.to_datetime(df['time'],unit='s')
    print(df)

Total positions= 2
      ticket                time       time_msc  time_update  time_update_msc  type   magic  identifier  reason  volume  price_open   sl   tp  price_current  swap  profit  symbol           comment external_id
0  273807416 2024-11-26 15:02:27  1732633347392   1732633347    1732633347392     0  234000   273807416       3     0.1     153.220  0.0  0.0        153.221   0.0    0.07  USDJPY  python script op            
1  273807474 2024-11-26 15:02:34  1732633354114   1732633354    1732633354114     0  234000   273807474       3     0.1     153.204  0.0  0.0        153.221   0.0    1.11  USDJPY  python script op            


In [ ]:
# create a close request
position_id=result.order
price=mt5.symbol_info_tick(symbol).bid
deviation=20
request={
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_SELL,
    "position": position_id,
    "price": price,
    "deviation": deviation,
    "magic": 234000,
    "comment": "python script close",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_RETURN,
}
# send a trading request
result=mt5.order_send(request)
print("3. close position #{}: sell {} {} lots at {} with deviation={} points".format(position_id,symbol,lot,price,deviation));
df=pd.DataFrame(list(result._asdict().items()),columns=['property','value'])
print(df)
if result.retcode != mt5.TRADE_RETCODE_DONE:
    print("4.1 order_send failed, retcode={}".format(result.retcode))
else:
    print("4.2 position #{} closed, {}".format(position_id,result))
    # request the result as a dictionary and display it element by element
    result_dict=result._asdict()
    for field in result_dict.keys():
        print("   {}={}".format(field,result_dict[field]))
        # if this is a trading request structure, display it element by element as well
        if field=="request":
            traderequest_dict=result_dict[field]._asdict()
            for tradereq_filed in traderequest_dict:
                print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))

3. close position #273807474: sell USDJPY 0.1 lots at 153.169 with deviation=20 points
            property                                              value
0            retcode                                              10030
1               deal                                                  0
2              order                                                  0
3             volume                                                0.0
4              price                                                0.0
5                bid                                                0.0
6                ask                                                0.0
7            comment                           Unsupported filling mode
8         request_id                                                  0
9   retcode_external                                                  0
10           request  (1, 234000, 0, USDJPY, 0.1, 153.169, 0.0, 0.0,...
4.1 order_send failed, retcode=10030


In [35]:
positions = mt5.positions_get()
df = pd.DataFrame(list(positions), columns=positions[0]._asdict().keys())
df['time'] = pd.to_datetime(df['time'], unit='s')
print(df)

      ticket                time       time_msc  time_update  time_update_msc  type   magic  identifier  reason  volume  price_open   sl   tp  price_current  swap  profit  symbol           comment external_id
0  273807416 2024-11-26 15:02:27  1732633347392   1732633347    1732633347392     0  234000   273807416       3     0.1     153.220  0.0  0.0        153.062   0.0  -10.32  USDJPY  python script op            
1  273807474 2024-11-26 15:02:34  1732633354114   1732633354    1732633354114     0  234000   273807474       3     0.1     153.204  0.0  0.0        153.062   0.0   -9.28  USDJPY  python script op            
2  273808661 2024-11-26 15:03:30  1732633410866   1732633410    1732633410866     1  234000   273808661       3     0.1     153.108  0.0  0.0        153.087   0.0    1.37  USDJPY  python script cl            


In [37]:
# close 1 position by position ticket

if len(positions)>0:
    position_id=positions[0].ticket
    symbol=positions[0].symbol
    price=mt5.symbol_info_tick(symbol).bid
    deviation=20
    request={
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL,
        "position": position_id,
        "price": price,
        "deviation": deviation,
        "magic": 234000,
        "comment": "python script close",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    # send a trading request
    result=mt5.order_send(request)
    print("5. close position #{}: sell {} {} lots at {} with deviation={} points".format(position_id,symbol,lot,price,deviation));
    df=pd.DataFrame(list(result._asdict().items()),columns=['property','value'])
    print(df)
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        print("6.1 order_send failed, retcode={}".format(result.retcode))
    else:
        print("6.2 position #{} closed, {}".format(position_id,result))
        # request the result as a dictionary and display it element by element
        result_dict=result._asdict()
        for field in result_dict.keys():
            print("   {}={}".format(field,result_dict[field]))
            # if this is a trading request structure, display it element by element as well
            if field=="request":
                traderequest_dict=result_dict[field]._asdict()
                for tradereq_filed in traderequest_dict:
                    print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))

5. close position #273807416: sell USDJPY 0.1 lots at 153.146 with deviation=20 points
            property                                              value
0            retcode                                              10009
1               deal                                          274341240
2              order                                          273813454
3             volume                                                0.1
4              price                                            153.146
5                bid                                            153.146
6                ask                                            153.175
7            comment                                   Request executed
8         request_id                                         1287851500
9   retcode_external                                                  0
10           request  (1, 234000, 0, USDJPY, 0.1, 153.146, 0.0, 0.0,...
6.2 position #273807416 closed, OrderSendResult(r

In [50]:
positions_total=mt5.positions_total()
if positions_total>0:
    print("Total positions=",positions_total)
    # get all positions on USDJPY
    positions=mt5.positions_get(symbol="USDJPY")
    df=pd.DataFrame(list(positions),columns=positions[0]._asdict().keys())
    df['time']=pd.to_datetime(df['time'],unit='s')
    print(df)

Total positions= 2
      ticket                time       time_msc  time_update  time_update_msc  type   magic  identifier  reason  volume  price_open   sl   tp  price_current  swap  profit  symbol           comment external_id
0  273807474 2024-11-26 15:02:34  1732633354114   1732633354    1732633354114     0  234000   273807474       3     0.1     153.204  0.0  0.0        153.275   0.0    4.63  USDJPY  python script op            
1  273808661 2024-11-26 15:03:30  1732633410866   1732633410    1732633410866     1  234000   273808661       3     0.1     153.108  0.0  0.0        153.302   0.0  -12.65  USDJPY  python script cl            


In [75]:
# get the number of orders in history
pd.set_option('display.max_columns', 500) # number of columns to be displayed
pd.set_option('display.width', 1500)      # max table width to display

from_date=datetime(2024,6,1) # year, month, day
to_date=datetime.now()
print(from_date, to_date)

from_timestamp = int(from_date.timestamp())
to_timestamp = int(to_date.timestamp())
print(from_timestamp, to_timestamp)

history_orders=mt5.history_orders_get(from_timestamp, to_timestamp)

if history_orders is None:
    print("No history orders")
if len(history_orders) > 0:
    data = []
    for order in history_orders:
        data.append(order._asdict())
    df = pd.DataFrame(data)
    df['time_done'] = pd.to_datetime(df['time_done'], unit='s')
    df['time_done_msc'] = pd.to_datetime(df['time_done'], unit='s')
    df['time_setup'] = pd.to_datetime(df['time_setup'], unit='s')
    df['time_setup_msc'] = pd.to_datetime(df['time_expiration'], unit='s')
    print(df)



2024-06-01 00:00:00 2024-11-26 20:50:57.814404
1717174800 1732629057
       ticket          time_setup time_setup_msc           time_done       time_done_msc  time_expiration  type  type_time  type_filling  state   magic  position_id  position_by_id  reason  volume_initial  volume_current  price_open       sl       tp  price_current  price_stoplimit  symbol           comment external_id
0   234954404 2024-07-29 14:49:27     1970-01-01 2024-07-29 14:49:27 2024-07-29 14:49:27                0     0          0             1      4  234000    234954404               0       3            0.10             0.0       0.000  153.574  153.974        153.774              0.0  USDJPY  python script op            
1   234954505 2024-07-29 14:49:44     1970-01-01 2024-07-29 14:49:44 2024-07-29 14:49:44                0     0          0             1      4  234000    234954505               0       3            0.10             0.0       0.000  153.572  153.972        153.773              0.0  USDJP

In [86]:
# close all positions
positions = mt5.positions_get()

if positions is None or len(positions) == 0:
    print("ไม่มีคำสั่งที่เปิดอยู่")
else:
    for position in positions:
        ticket = position.ticket  # หมายเลขคำสั่งที่เปิด
        symbol = position.symbol  # ชื่อสัญลักษณ์ (เช่น EURUSD, USDJPY)
        volume = position.volume  # ปริมาณของตำแหน่ง
        price = mt5.symbol_info_tick(symbol).ask  # ใช้ราคา ask ปัจจุบันในการปิดตำแหน่ง

        # สร้างคำขอการปิดคำสั่ง
        close_request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "position": ticket,
            "price": price,
            "deviation": 20,  # กำหนดระดับความเบี่ยงเบนของราคา
            "magic": 234000,  # รหัส EA (หากใช้)
            "comment": "ปิดตำแหน่งทั้งหมด",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

        # ส่งคำขอการปิดตำแหน่ง
        result = mt5.order_send(close_request)

        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"ไม่สามารถปิดตำแหน่ง {ticket}: {result.comment}, retcode={result.retcode}")
        else:
            print(f"ตำแหน่ง {ticket} ปิดสำเร็จ")

ตำแหน่ง 273808661 ปิดสำเร็จ
